In [1]:
import numpy as np
import gdspy
import os
import cv2
import matplotlib.pyplot as plt
from shapely.geometry import Polygon
import shapely
import tqdm
import matplotlib.image
from geneticalgorithm import geneticalgorithm as ga
from slice_utilities import poly_intersection, slice_gds, cost_reward

In [2]:
chip_r = gdspy.GdsLibrary(infile='metal_slice.gds')


In [3]:
K=chip_r.cells.keys()


In [4]:
top_io = chip_r.cells['top']

In [5]:
#Given a set of polygons this function returns a list of list each of which is a connected set of polygons.
def signals_per_layer(polyset):
    N=len(polyset)
    #Building a polygon list 
    P=list()
    for n in range(N):
        P.append(Polygon(polyset[n]))
    A = np.zeros((N,N), dtype=bool)
    for n in range(N):
        A[n,n]=True
        for m in range(n+1,N):
            A[n,m] = P[n].intersects(P[m])
            A[m,n] = A[n,m]
    Signals=list()
    for n in range(N):
        if sum(A[n,:])>0: #indicating this cell has not been used
            signal=list() #creating a new signal
            signal.append(n) # appending the current polygon 
            A[n,n]=False # indecating the the polygon has been added to sigmal
            A[:,n]=False # removing the cell from 
            ind_M =A[n,:].nonzero()[0].tolist()
            A[n,:]=False 
            for m in ind_M:
                signal.append(m)
                A[m,m]=False
                C=A[:,m].nonzero()[0]
                for c in C:
                    ind_c =A[:,c].nonzero()[0].tolist()
                    for i in ind_c:
                        signal.append(i)
                        A[i,i]=False
            signal=list(set(signal))
            Signals.append(signal)
    return Signals
                        
                    
            
        
    

In [6]:
# This cell will create signals per layer each such 
# signal will be a unique cell keyed by naumber of 
# layer and signal number
cells_keys = list(chip_r.cells.keys())
cells_keys.remove('top')
metal_layers = [61,62,63,64,65,66]
via_layers = [70,71,72,73,74]
signal_num=0
top_cell = gdspy.Cell('top_metal')
sections=[]
for key in cells_keys:
    layer=eval(key)[0]
    if layer in metal_layers:
        cell = chip_r.cells[key]
        poly = cell.get_polygons()
        S = signals_per_layer(poly)
        for s in S:
            signal_cell = gdspy.Cell(str((layer, signal_num)))
#             print(len(s),signal_num)
            for p in s:
                signal_cell.add(gdspy.Polygon(poly[p], layer=layer, datatype=signal_num))
                top_cell.add(gdspy.Polygon(poly[p], layer=layer, datatype=signal_num))
            signal_num +=1
            sections.append(signal_cell)
    if layer in via_layers:
        cell = chip_r.cells[key]
        sections.append(cell)
        
sections.append(top_cell)
gds_lib = gdspy.GdsLibrary()
gds_lib.write_gds('metal_slice_signals_layer.gds', cells=sections)


In [7]:
len(sections)

395

This section will combine signals with the via

In [8]:
#This function returns true if the cell is connected to the via polygon 
def check_via_metal(poly,sig_poly):
    p1 = Polygon(poly)
    for i in range(len(sig_poly)):
        p2 = Polygon(sig_poly[i])
        if p2.intersects(p1):
            return True
    return False
    

In [9]:
#The cell computes the SIGMALS list with for every cell in the via cells compute the asociated signals 
chip_signals = gdspy.GdsLibrary(infile='metal_slice_signals_layer.gds')
cells_keys = list(chip_signals.cells.keys())
SIGNALS=list()
for i in via_layers:
    via_cell = chip_signals.cells[str((i, 0))]
    polygons = via_cell.get_polygons()
    for p in range(len(polygons)):
#         print(p ,i )
        poly = polygons[p]
        signal_list=list()
        for key in cells_keys:
            if key=='top_metal':
                pass
            else:              
                layer_num,signal_num = eval(key)
                if layer_num in metal_layers:
                    sig_poly = chip_signals.cells[key].get_polygons()
                    res = check_via_metal(poly,sig_poly)
                    if res:
                        print('Found')
                        signal_list.append(signal_num)
        if len(signal_list)>0:
            print(signal_list)
            SIGNALS.append(signal_list)
                    
        
    

Found
Found
Found
Found
Found
[25, 55, 115, 233, 357]
Found
Found
Found
Found
Found
[16, 20, 54, 126, 376]
Found
Found
Found
[52, 127, 376]
Found
Found
Found
[51, 130, 210]
Found
Found
Found
Found
[50, 131, 226, 356]
Found
Found
Found
Found
[49, 133, 260, 367]
Found
Found
Found
Found
[49, 134, 260, 367]
Found
Found
Found
[48, 135, 222]
Found
Found
Found
[48, 136, 222]
Found
Found
Found
Found
[47, 139, 226, 356]
Found
Found
[47, 140]
Found
Found
Found
Found
[45, 46, 142, 256]
Found
Found
Found
[45, 143, 206]
Found
Found
[43, 118]
Found
Found
Found
Found
Found
[43, 44, 191, 211, 212]
Found
Found
Found
[90, 144, 145]
Found
Found
Found
Found
Found
Found
[5, 104, 192, 213, 292, 295]
Found
Found
Found
Found
[42, 147, 148, 234]
Found
Found
Found
Found
[42, 149, 234, 373]
Found
Found
[41, 151]
Found
Found
[41, 152]
Found
Found
Found
Found
[40, 160, 220, 362]
Found
Found
Found
[39, 193, 223]
Found
Found
[38, 164]
Found
Found
[38, 165]
Found
Found
Found
[37, 171, 208]
Found
Found
[37, 188]
Found

[150, 215]
Found
Found
Found
Found
[36, 152, 216, 377]
Found
Found
Found
Found
Found
[1, 36, 153, 216, 377]
Found
Found
Found
[1, 153, 217]
Found
Found
Found
Found
[33, 154, 218, 360]
Found
Found
[155, 218]
Found
Found
Found
Found
Found
[16, 20, 156, 219, 276]
Found
Found
Found
[157, 220, 362]
Found
Found
[158, 221]
Found
Found
Found
Found
[36, 162, 224, 377]
Found
Found
Found
Found
[36, 163, 224, 377]
Found
Found
Found
Found
[47, 166, 226, 356]
Found
Found
Found
Found
Found
Found
Found
[25, 88, 98, 100, 101, 170, 227]
Found
Found
Found
[33, 122, 228]
Found
Found
Found
[175, 231, 357]
Found
Found
[178, 239]
Found
Found
[179, 243]
Found
Found
Found
Found
[180, 244, 285, 286]
Found
Found
Found
Found
Found
[181, 245, 328, 329, 330]
Found
Found
Found
Found
[182, 183, 247, 358]
Found
Found
Found
[6, 185, 253]
Found
Found
Found
[186, 260, 368]
Found
Found
Found
[35, 187, 256]
Found
Found
Found
Found
Found
Found
Found
Found
Found
Found
Found
[0, 25, 56, 62, 65, 76, 94, 199, 264, 324, 385]
Fou

Found
[0, 25, 56, 62, 65, 76, 94, 199, 264, 324, 385]
Found
Found
Found
Found
Found
Found
Found
Found
Found
Found
Found
[0, 25, 56, 62, 65, 76, 94, 199, 264, 324, 385]
Found
Found
Found
Found
Found
Found
Found
Found
Found
Found
Found
[0, 25, 56, 62, 65, 76, 94, 199, 264, 324, 385]
Found
Found
Found
Found
Found
Found
Found
Found
Found
Found
Found
[0, 25, 56, 62, 65, 76, 94, 199, 264, 324, 385]
Found
Found
Found
Found
Found
Found
Found
Found
Found
Found
Found
[0, 25, 56, 62, 65, 76, 94, 199, 264, 324, 385]
Found
Found
Found
Found
Found
Found
Found
Found
Found
Found
Found
[0, 25, 56, 62, 65, 76, 94, 199, 264, 324, 385]
Found
Found
Found
Found
Found
Found
Found
Found
Found
Found
Found
[0, 25, 56, 62, 65, 76, 94, 199, 264, 324, 385]
Found
Found
Found
Found
Found
Found
Found
Found
Found
Found
Found
[0, 25, 56, 62, 65, 76, 94, 199, 264, 324, 385]
Found
Found
Found
Found
Found
Found
Found
Found
Found
Found
Found
[0, 25, 56, 62, 65, 76, 94, 199, 264, 324, 385]
Found
Found
Found
Found
Found
Foun

Found
Found
[267, 331]
Found
Found
Found
[53, 267, 331]
Found
Found
[267, 331]
Found
Found
Found
[53, 267, 331]
Found
Found
[267, 331]
Found
Found
Found
[53, 267, 331]
Found
Found
[267, 331]
Found
Found
Found
[53, 267, 332]
Found
Found
[267, 332]
Found
Found
Found
[53, 267, 332]
Found
Found
[267, 332]
Found
Found
Found
[53, 267, 332]
Found
Found
[267, 332]
Found
Found
Found
[53, 267, 333]
Found
Found
[267, 333]
Found
Found
Found
[53, 267, 333]
Found
Found
[267, 333]
Found
Found
Found
[53, 267, 334]
Found
Found
[267, 334]
Found
Found
Found
[53, 267, 335]
Found
Found
[267, 335]
Found
Found
Found
[53, 267, 336]
Found
Found
[267, 336]
Found
Found
Found
[53, 267, 337]
Found
Found
[267, 337]
Found
Found
Found
[53, 267, 337]
Found
Found
[267, 337]
Found
Found
Found
[53, 267, 338]
Found
Found
[267, 338]
Found
Found
Found
[53, 267, 338]
Found
Found
[267, 338]
Found
Found
Found
[53, 267, 338]
Found
Found
[267, 338]
Found
Found
Found
[53, 267, 339]
Found
Found
[267, 339]
Found
Found
Found
[53, 26

Found
Found
Found
Found
[0, 24, 29, 200, 265, 325, 386]
Found
Found
Found
Found
Found
Found
Found
[0, 24, 29, 200, 265, 325, 386]
Found
Found
Found
Found
Found
Found
Found
[1, 16, 20, 201, 266, 326, 387]
Found
Found
Found
Found
Found
Found
Found
[1, 16, 20, 201, 266, 326, 387]
Found
Found
Found
Found
Found
Found
Found
[1, 16, 20, 201, 266, 326, 387]
Found
Found
Found
Found
Found
Found
Found
[1, 16, 20, 201, 266, 326, 387]
Found
Found
Found
Found
Found
Found
Found
[1, 16, 20, 201, 266, 326, 387]
Found
Found
Found
Found
Found
Found
Found
[1, 16, 20, 201, 266, 326, 387]
Found
Found
Found
Found
Found
Found
Found
[1, 16, 20, 201, 266, 326, 387]
Found
Found
Found
Found
Found
Found
Found
[1, 16, 20, 201, 266, 326, 387]
Found
Found
Found
Found
Found
Found
Found
[1, 16, 20, 201, 266, 326, 387]
Found
Found
Found
Found
Found
Found
Found
[1, 16, 20, 201, 266, 326, 387]
Found
Found
Found
Found
Found
Found
Found
[1, 16, 20, 201, 266, 326, 387]
Found
Found
Found
Found
Found
Found
Found
[1, 16, 20, 20

In [10]:
for S in SIGNALS:
    print(len(S))


5
5
3
3
4
4
4
3
3
4
2
4
3
2
5
3
6
4
4
2
2
4
3
2
2
3
2
4
3
3
2
2
4
6
6
3
2
3
4
3
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
3
5
6
2
2
3
4
5
3
6
6
3
3
4
6
4
4
4
3
2
3
3
2
2
3
3
2
4
5
3
4
2
5
3
2
4
4
4
7
3
3
2
2
4
5
4
3
3
3
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
6
3
4
3
4
2
3
2
4
4
3
3
3
4
4
3
6
4
5
8
3
3
3
3
4
5
3
7
3
2
4
4
6
4
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
5
4
5
4
5
4
5
4
5
4
5
4
5
4
5
4
5
4
5
4
5
4
5
4
5
4
5
4
7
7
7
7
7
7
7
7
7
6
5
5
6
5
4
4
5
5
4
4
5
5
4
4
4
4
4
4
4
4
5
4
5
4
5
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3

In [11]:
len(SIGNALS)

700

In [12]:
v0=list()
for S in SIGNALS:
    if 0 in S:
        print(S)
        for s in S:
            v0.append(s)
        v0=list(set(v0))
print(len(v0))
print(v0)

[0, 25, 56, 62, 65, 76, 94, 199, 264, 324, 385]
[0, 25, 56, 62, 65, 76, 94, 199, 264, 324, 385]
[0, 25, 56, 62, 65, 76, 94, 199, 264, 324, 385]
[0, 25, 56, 62, 65, 76, 94, 199, 264, 324, 385]
[0, 25, 56, 62, 65, 76, 94, 199, 264, 324, 385]
[0, 25, 56, 62, 65, 76, 94, 199, 264, 324, 385]
[0, 25, 56, 62, 65, 76, 94, 199, 264, 324, 385]
[0, 25, 56, 62, 65, 76, 94, 199, 264, 324, 385]
[0, 25, 56, 62, 65, 76, 94, 199, 264, 324, 385]
[0, 25, 56, 62, 65, 76, 94, 199, 264, 324, 385]
[0, 25, 56, 62, 65, 76, 94, 199, 264, 324, 385]
[0, 25, 56, 62, 65, 76, 94, 199, 264, 324, 385]
[0, 25, 56, 62, 65, 76, 94, 199, 264, 324, 385]
[0, 25, 56, 62, 65, 76, 94, 199, 264, 324, 385]
[0, 25, 56, 62, 65, 76, 94, 199, 264, 324, 385]
[0, 25, 56, 62, 65, 76, 94, 199, 264, 324, 385]
[0, 25, 56, 62, 65, 76, 94, 199, 264, 324, 385]
[0, 25, 56, 62, 65, 76, 94, 199, 264, 324, 385]
[0, 25, 56, 62, 65, 76, 94, 199, 264, 324, 385]
[0, 24, 29, 200, 265, 325, 386]
[0, 24, 29, 200, 265, 325, 386]
[0, 24, 29, 200, 265, 32

In [13]:
N_singals = 0
for key in cells_keys:
    if key=='top_metal':
        pass
    else:              
        layer_num,signal_num= eval(key)
        N_singals=max(N_singals,signal_num)
N_singals+=1

In [14]:
key

'top_metal'

Building the signals connectivity matrix

In [15]:
As = np.zeros((N_singals,N_singals), dtype=bool)
for s in SIGNALS:
    for n in s:
        As[n,n]=True
        for m in s:
            As[n,m]=True
            As[m,n]=True

In [16]:
sum(As[0,:])

23

In [17]:
N = N_singals
Signals_c=list()
for n in range(N):
    if sum(As[n,:])>0: #indicating this cell has not been used
        signal=list() #creating a new signal
        signal.append(n) # appending the current polygon 
        As[n,n]=False # indecating the the polygon has been added to sigmal
        As[:,n]=False # removing the cell from 
        ind_M =As[n,:].nonzero()[0].tolist()
        As[n,:]=False 
        for m in ind_M:
            signal.append(m)
            As[n,m]=False
            As[m,n]=False
            C=As[:,m].nonzero()[0]
            for c in C:
                ind_c =As[:,c].nonzero()[0].tolist()
                for i in ind_c:
                    signal.append(i)
                    As[:,i]=False
                    As[i,:]=False
        signal=list(set(signal))
        Signals_c.append(signal)

In [18]:
for v in Signals_c:
    print(min(v),len(v))

0 97
2 10
3 4
4 12
5 15
6 11
9 3
11 9
12 10
13 6
15 5
21 4
22 4
23 4
30 6
31 4
32 4
33 12
34 3
37 16
38 3
40 4
41 3
43 6
49 3
50 6
51 4
52 5
106 3
109 2
113 3
117 1
119 3
121 1
137 1
150 3
155 1
156 2
157 2
158 2
162 5
172 2
175 2
178 2
179 2
182 3
193 2
230 2
248 2
250 5
271 1
272 2
274 1
298 2
299 1
300 1
305 2
306 1
316 1
364 1
370 1
374 1


In [19]:
chip_signals = gdspy.GdsLibrary(infile='metal_slice_signals_layer.gds')
gds_lib = gdspy.GdsLibrary()
cells_keys = list(chip_signals.cells.keys())
cells_triple=list()
for key in cells_keys:
    if key=='top_metal':
        cell = chip_signals.cells[key]
        cells_triple.append(cell)
    else:              
        layer_num,signal_num = eval(key)
        if layer_num in metal_layers:
            for s in Signals_c:
                if (signal_num in s):
                    s.sort()
                    name = str((layer_num,signal_num,s[0]))
                    print(name)
                    cell = chip_signals.cells[key]
                    cell.name = name
                    cells_triple.append(cell)

        else:
            cell = chip_signals.cells[key]
            cells_triple.append(cell)
gds_lib.write_gds('metal_slice_signals_unique.gds', cells=cells_triple)
            

(66, 0, 0)
(66, 1, 0)
(66, 2, 2)
(66, 3, 3)
(66, 4, 4)
(66, 5, 5)
(66, 6, 6)
(66, 7, 6)
(66, 8, 6)
(66, 9, 9)
(66, 10, 0)
(66, 11, 11)
(66, 12, 12)
(61, 13, 13)
(61, 15, 15)
(61, 16, 0)
(61, 17, 13)
(61, 19, 15)
(61, 20, 0)
(61, 21, 21)
(61, 22, 22)
(61, 23, 23)
(61, 24, 0)
(61, 25, 0)
(61, 26, 21)
(61, 27, 22)
(61, 28, 23)
(61, 29, 0)
(61, 30, 30)
(61, 31, 31)
(61, 32, 32)
(61, 33, 33)
(61, 34, 34)
(61, 35, 30)
(61, 36, 0)
(61, 37, 37)
(61, 38, 38)
(61, 39, 0)
(61, 40, 40)
(61, 41, 41)
(61, 42, 11)
(61, 43, 43)
(61, 44, 43)
(61, 45, 12)
(61, 46, 12)
(61, 47, 37)
(61, 48, 2)
(61, 49, 49)
(61, 50, 50)
(61, 51, 51)
(61, 52, 52)
(61, 53, 0)
(61, 54, 0)
(61, 55, 0)
(61, 56, 0)
(61, 62, 0)
(61, 64, 0)
(61, 65, 0)
(61, 68, 13)
(61, 74, 0)
(61, 76, 0)
(61, 84, 13)
(61, 88, 0)
(61, 90, 5)
(61, 94, 0)
(61, 95, 0)
(61, 96, 0)
(61, 98, 0)
(61, 100, 0)
(61, 101, 0)
(61, 104, 5)
(61, 106, 106)
(61, 109, 109)
(61, 111, 0)
(61, 113, 113)
(62, 115, 0)
(62, 116, 52)
(62, 117, 117)
(62, 118, 43)
(62, 11

unify the cells under the signals

In [20]:
import numpy as np
import gdspy
import os
import cv2
import matplotlib.pyplot as plt
from shapely.geometry import Polygon
import shapely
import tqdm
import matplotlib.image
from geneticalgorithm import geneticalgorithm as ga
from slice_utilities import poly_intersection, slice_gds, cost_reward

In [21]:
metal_layers = [61,62,63,64,65,66]
chip_signals = gdspy.GdsLibrary(infile='metal_slice_signals_unique.gds')
cells_keys = list(chip_signals.cells.keys())
cells_signals=list()
uni_sig=list()
check_cell = gdspy.Cell('check')
for key in cells_keys:
    if key=='top_metal':
        cell = chip_signals.cells[key]
        cells_signals.append(cell)
        cells_signals.append(check_cell)
    elif(len(eval(key))>2):
        layer_num,signal_num,unique_signals = eval(key)
        if unique_signals in uni_sig:
            pass
        else:
            signal_cell = gdspy.Cell(str(unique_signals))
            uni_sig.append(unique_signals)
            if layer_num in metal_layers:
#                 cell = chip_signals.cells[key]
                for key2 in cells_keys:
                    if key2=='top_metal':
                        pass
                    elif(len(eval(key2))>2):
                        layer_num2,signal_num2,unique_signals2 = eval(key2)
                        if(unique_signals==unique_signals2):
                            poly = chip_signals.cells[key2].get_polygons()
                            for p in poly:
                                print('adding')
                                signal_cell.add(gdspy.Polygon(p))
                                check_cell.add(gdspy.Polygon(p))
#             cell.name=str(unique_signals)                            
            cells_signals.append(signal_cell)
gds_lib = gdspy.GdsLibrary()
gds_lib.write_gds('metal_slice_signals_unique_all.gds', cells=cells_signals)        


                    


adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding

In [22]:
metal_layers = [61,62,63,64,65,66]
chip_signals = gdspy.GdsLibrary(infile='metal_slice_signals_unique.gds')
cells_keys = list(chip_signals.cells.keys())
cells_signals=list()
uni_sig=list()
check_cell = gdspy.Cell('check')
for key in cells_keys:
    if key=='top_metal':
        cell = chip_signals.cells[key]
        cells_signals.append(cell)
        cells_signals.append(check_cell)
    elif(len(eval(key))>2):
        layer_num,signal_num,unique_signals = eval(key)
        if [layer_num, unique_signals] in uni_sig:
            pass
        else:
            signal_cell = gdspy.Cell(str((layer_num,unique_signals)))
            uni_sig.append([layer_num, unique_signals])
            for key2 in cells_keys:
                if key2=='top_metal':
                    pass
                elif(len(eval(key2))>2):
                    layer_num2,signal_num2,unique_signals2 = eval(key2)
                    if(layer_num==layer_num2 and unique_signals==unique_signals2):
                        poly = chip_signals.cells[key2].get_polygons()
                        for p in poly:
                            print('adding')
                            signal_cell.add(gdspy.Polygon(p))
                            check_cell.add(gdspy.Polygon(p))
#             cell.name=str(unique_signals)                            
            cells_signals.append(signal_cell)
gds_lib = gdspy.GdsLibrary()
gds_lib.write_gds('metal_slice_signals_unique_all_layer.gds', cells=cells_signals)        


                    


ValueError: [GDSPY] Cell named check already present in library.

Setting up the GA alg

In [ ]:
def reward_by_signals(file,x_min,y_min,x_max,y_max,res,layers,signal_num):
    layers.sort()
    w = x_max - x_min
    h = y_max - y_min
#     mask_reward = np.zeros((w*res,h*res))
    mask_reward_list = list()
    for l in layers:
        mask_reward_list.append(np.zeros((w*res,h*res)))
    keys = list(chip_signals.cells.keys())
    keys.remove('check1')
    keys.remove('top_metal')
    min_layer = min(layers)+1
    max_layer = max(layers)+1
    for layer in layers:
        e_key = str((layer,signal_num))
        r = (layer -min_layer+1) /(max_layer-min_layer)
        if e_key in keys:
            polygons = file.cells[e_key].get_polygons()
            for p in polygons:
                ver=[]
                for i in range(len(p)):
                    ver.append([res*(p[i,1]-y_min), res*(p[i,0]-x_min)])
                ver = np.array(ver,dtype=int)
                for i in range(len(layers)):
                    if(layers[i]==layer): 
                        cv2.fillPoly(mask_reward_list[i], pts=[ver], color=r)
    return mask_reward_list


    

In [ ]:
def cost_by_signals(file,x_min,y_min,x_max,y_max,res,layers,signals_num):
    layers.sort()
    w = x_max - x_min
    h = y_max - y_min
#     mask_reward = np.zeros((w*res,h*res))
    cost_reward_list = list()
    for l in layers:
        cost_reward_list.append(np.zeros((w*res,h*res)))
    keys = list(chip_signals.cells.keys())
    keys.remove('check1')
    keys.remove('top_metal')
    min_layer = min(layers)+1
    max_layer = max(layers)+1
    for key in keys:
        layer,sig_num = eval(key)
        r = (layer -min_layer+1) /(max_layer-min_layer)
        if(sig_num in signals_num):
            pass
        else:
            polygons = file.cells[key].get_polygons()
            for p in polygons:
                ver=[]
                for i in range(len(p)):
                    ver.append([res*(p[i,1]-y_min), res*(p[i,0]-x_min)])
                ver = np.array(ver,dtype=int)
                for i in range(len(layers)):
                    if(layers[i]==layer): 
                        cv2.fillPoly(cost_reward_list[i], pts=[ver], color=r)   
    return cost_reward_list

In [ ]:
name_of_file = 'metal_slice_signals_unique_all_layer.gds'
chip_signals = gdspy.GdsLibrary(infile=name_of_file)
bounding_box = [(1200, 730),(1200, 775),(1390, 775),(1390,762), (1210, 762), (1210, 730)]
x = list()
y = list()
for i in range(len(bounding_box)):
    x.append(bounding_box[i][0])
    y.append(bounding_box[i][1])
x_min = min(x)
y_min = min(y)
x_max = max(x)
y_max = max(y)
signal_num_1 = 0
signal_num_2 = 5
metal_layers = [61,62,63,64,65,66]
resolution = 100
reward_mask1 = reward_by_signals(chip_signals,x_min,y_min,x_max,y_max,resolution,metal_layers,signal_num_1)
reward_mask2 = reward_by_signals(chip_signals,x_min,y_min,x_max,y_max,resolution,metal_layers,signal_num_2)
cost_mask = cost_by_signals(chip_signals,x_min,y_min,x_max,y_max,resolution,metal_layers,[signal_num_1,signal_num_2])


In [ ]:
for i in range(len(cost_mask)):
    matplotlib.image.imsave('mask_cost_'+str(metal_layers[i])+'.png', cost_mask[i]) 
    matplotlib.image.imsave('mask_reward_1_'+str(metal_layers[i])+'.png', reward_mask1[i]) 
    matplotlib.image.imsave('mask_reward_2_'+str(metal_layers[i])+'.png', reward_mask2[i]) 


In [ ]:
W,H = cost_mask[0].shape

In [ ]:
def cost_reward(x,y,l,s):
    ind_l=(metal_layers).index(l)
    cost= 0
    for i in range(ind_l,len(metal_layers)):
            cost += sum(sum(cost_mask[i][(x-20):(x+20),(y-20):(y+20)]))
    reward = 0
    if s==1:
        reward = sum(sum(reward_mask1[i][(x-20):(x+20),(y-20):(y+20)]))
    elif s==2:
        reward = sum(sum(reward_mask2[i][(x-20):(x+20),(y-20):(y+20)]))
    
    return cost,reward
    


In [ ]:
len(metal_layers)

In [ ]:
def cost_func(X):
    x1=int(X[0])
    y1=int(X[1])
    l1=int(X[2])
    x2=int(X[3])
    y2=int(X[4])
    l2=int(X[5])
    cost1, reward1 = cost_reward(x1,y1,l1,1)
    cost2, reward2 = cost_reward(x2,y2,l2,2)
    cost = cost1+cost2
    reward = 100*(reward1+reward2)
    L_cost = (x1-x2)**2+(y1-y2)**2
    alpha=0.01
    if (cost>0 or reward1==0 or reward2==0):
        return cost
    else:
        return -reward + alpha*L_cost

In [ ]:
algorithm_param = {'max_num_iteration': 100,\
                   'population_size':10000,\
                   'mutation_probability':0.1,\
                   'elit_ratio': 0.01,\
                   'crossover_probability': 0.5,\
                   'parents_portion': 0.3,\
                   'crossover_type':'uniform',\
                   'max_iteration_without_improv':None}

In [ ]:
varbound=np.array([[20,W-20],[20,H-20],[61,66],[20,W-20],[20,H-20],[61,66]])
vartype=np.array([['real'],['real'],['int'],['real'],['real'],['int']])
model=ga(function=cost_func,dimension=6,variable_type_mixed=vartype,variable_boundaries=varbound,algorithm_parameters=algorithm_param)
model.run()

In [ ]:
chip_r = gdspy.GdsLibrary(infile='metal_slice_signals_unique_all.gds')

x1,y1,l1,x2,y2,l2 = model.best_variable
l1 = int(l1)
l2 = int(l2)
x1_g=x1/resolution+x_min
y1_g=y1/resolution+y_min
x2_g=x2/resolution+x_min
y2_g=y2/resolution+y_min

r=20/resolution
res_poly1=[[x1_g-r, y1_g-r],
    [x1_g-r, y1_g+r],
    [x1_g+r, y1_g+r],
    [x1_g+r, y1_g-r],
    [x1_g-r, y1_g-r]]

res_poly2=[[x2_g-r, y2_g-r],
    [x2_g-r, y2_g+r],
    [x2_g+r, y2_g+r],
    [x2_g+r, y2_g-r],
    [x2_g-r, y2_g-r]]
chip_r.cells['0'].add(gdspy.Polygon(res_poly1,layer=80,datatype=555))
chip_r.cells['5'].add(gdspy.Polygon(res_poly2,layer=80,datatype=555))
chip_r.cells['top_metal'].add(gdspy.Polygon(res_poly1,layer=80,datatype=555))
chip_r.cells['top_metal'].add(gdspy.Polygon(res_poly2,layer=80,datatype=555))

gds_lib.write_gds('metal_slice_signals_unique_all_res.gds', chip_r)

In [ ]:
gdspy.LayoutViewer()

In [ ]:
l1

In [ ]:
l2

In [ ]:
res_poly2

In [ ]:
from EDA_class import EDA

In [ ]:
eda_class = EDA('130nm_edit_0001.gds','top_io')